In [6]:
import os
import numpy as np
import pandas as pd

## <p style='text-align: center; text-decoration: underline; color: #10A0B4;'> Datos oficiales de la **Union Europea** </p>
[Origen de los datos](https://www.eea.europa.eu/en/datahub/datahubitem-view/fa8b1229-3db6-495d-b18e-9c9b3267c02b)

---

## Autos registrados desde **2010** hasta **2024**

---

# <p style='text-align: center; color: orange;'>Normativa euro 6</p>

# <p style='text-align: center; text-decoration: underline; color: green;'>Limpiado de datos</p>
#### El dataset contiene mas de **14 Gigas** de informacion (69.643.588 filas).

El gran tamano del conjunto de datos trae problemas:
- No puedo subirlo a GitHub (+100MB). 
- Lento procesamiento.

Vamos a resolver ambas cuentiones achicando el dataset de manera tal que el impacto por usar menos informacion sea minimo

In [7]:
url = 'ignore_data.csv'
file_weight = os.path.getsize(url)
print('Database size: ', round((file_weight/1024)/1024/1024, 2), 'GB')

Database size:  14.07 GB


---

In [8]:
# Obtenemos las columnas del Data Frame
df_columns = pd.read_csv('ignore_data2.csv', nrows=0)
print('Columnas del data frame crudo: \n', df_columns.columns)

# Cargamos unicamente las que nos interesan
df_columns.drop(columns=['year', 'Country', 'Date of registration', 'Enedc (g/km)', 'r', 'VFN', 'Mp', 'Man', 'Tan', 'T', 'Va', 'Ve', 'MMS', 'Ct', 'Cr', 'Mt', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Fm', 'IT', 'Ernedc (g/km)', 'Erwltp (g/km)', 'De','Vf', 'Status', 'ech', 'RLFI', 'Mh', 'z (Wh/km)', 'ep (KW)', 'Mk', 'Cn', 'Electric range (km)'], inplace=True)
columns_to_load = ['ID', 'm (kg)', 'Ewltp (g/km)', 'Ft', 'ec (cm3)', 'Fuel consumption ']

# Separamos la informacion en fragmentos de 1 millon para que la computadora no se sobresature
df_crud = pd.read_csv('ignore_data2.csv', chunksize=1000000, usecols=columns_to_load)

print('\n\nColumnas que nos sirven para el proyecto: \n', df_columns.columns)

Columnas del data frame crudo: 
 Index(['ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va',
       'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)',
       'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm',
       'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT', 'Ernedc (g/km)',
       'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
       'Fuel consumption ', 'ech', 'RLFI', 'Electric range (km)'],
      dtype='object')


Columnas que nos sirven para el proyecto: 
 Index(['ID', 'm (kg)', 'Ewltp (g/km)', 'Ft', 'ec (cm3)', 'Fuel consumption '], dtype='object')


In [9]:
# Creamos el dataframe que utilisaremos
df = pd.DataFrame(columns=df_columns.columns)

# Cambiamos de nombre algunas columnas
rename_columns = {'m (kg)': 'Mass_(kg)', 'Ewltp (g/km)': 'CO2_emission_(g/km)', 'Ft': 'Fuel_type', 'ec (cm3)': 'Engine_size_cm3', 'Fuel consumption ': 'Fuel_consumption_(l/100km)'}
df.rename(columns=rename_columns, inplace=True)

for chunk in df_crud:

    chunk.rename(columns=rename_columns, inplace=True)    
    chunk = chunk.dropna()
    
    df = pd.concat([df, chunk], ignore_index=True)
    
print(df.shape)

# Lo guardo en un archivo aparte para evitar volver a ejecutar esta celda
df.to_csv('ignore_df_data_clean.csv', index=False)

C:\Users\matia\AppData\Local\Temp\ipykernel_22028\1486211447.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, chunk], ignore_index=True)


(23869031, 6)


---
---
---

### Limpiando los datos

In [10]:
df1 = pd.read_csv('ignore_df_data_clean.csv')
print('Tamano del archivo df1:', (round((os.path.getsize('ignore_df_data_clean.csv')/1024)/1024, 2)), 'MB')
df1.shape

Tamano del archivo df1: 948.15 MB


(23869031, 6)

In [11]:
#Tipos de combustibles
print(df1['Fuel_type'].value_counts())

for fuel_type in df1['Fuel_type'].unique():
    print(fuel_type, len(fuel_type))

Fuel_type
petrol             11045892
PETROL              4334156
diesel              3570807
DIESEL              1891905
petrol/electric     1602540
lpg                  529183
PETROL/ELECTRIC      451775
LPG                  164343
e85                   84993
diesel/electric       68062
DIESEL/ELECTRIC       55721
NG                    34385
ng                    24888
E85                    5969
NG-BIOMETHANE          4408
unknown                   2
ELECTRIC                  1
UNKNOWN                   1
Name: count, dtype: int64
PETROL 6
PETROL/ELECTRIC 15
DIESEL 6
LPG 3
NG-BIOMETHANE 13
DIESEL/ELECTRIC 15
NG 2
E85 3
UNKNOWN 7
ELECTRIC 8
petrol/electric 15
petrol 6
diesel 6
diesel/electric 15
lpg 3
ng 2
unknown 7
e85 3


In [12]:
#Todo en minuscula y sin espacios
df1['Fuel_type'] = df1['Fuel_type'].str.lower().replace(' ', '').apply(lambda x: x.strip())

#Eliminamos los que no se sabe su combustible
word_to_delete = ['unknown']
def follow_words(text, words):
    return any(word in str(text) for word in words)

df1 = df1[~df1.applymap(lambda x: follow_words(x, word_to_delete)).any(axis=1)]

df1['Fuel_type'].value_counts()

C:\Users\matia\AppData\Local\Temp\ipykernel_22028\3229457277.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df1 = df1[~df1.applymap(lambda x: follow_words(x, word_to_delete)).any(axis=1)]


Fuel_type
petrol             15380048
diesel              5462712
petrol/electric     2054315
lpg                  693526
diesel/electric      123783
e85                   90962
ng                    59273
ng-biomethane          4408
electric                  1
Name: count, dtype: int64

### +85% de los vehiculos usa nafta o diesel
Unicamente vamos a utilizar los que usan nafta y diesel

In [13]:
fuel_type_count = dict(df1[['Fuel_type']].value_counts())
print(fuel_type_count)

fuel_type_count = {str(key[0]): value for key, value in fuel_type_count.items()}
print(fuel_type_count)

fuel_type_count = {key: round((value/df1.shape[0])*100, 2) for key, value in fuel_type_count.items()}
print('\n', fuel_type_count)

#Porcentaje de uso de cada combustible
df1 = df1[df1['Fuel_type'].isin(['petrol', 'diesel'])]
df1[['Fuel_type']].value_counts()

{('petrol',): np.int64(15380048), ('diesel',): np.int64(5462712), ('petrol/electric',): np.int64(2054315), ('lpg',): np.int64(693526), ('diesel/electric',): np.int64(123783), ('e85',): np.int64(90962), ('ng',): np.int64(59273), ('ng-biomethane',): np.int64(4408), ('electric',): np.int64(1)}
{'petrol': np.int64(15380048), 'diesel': np.int64(5462712), 'petrol/electric': np.int64(2054315), 'lpg': np.int64(693526), 'diesel/electric': np.int64(123783), 'e85': np.int64(90962), 'ng': np.int64(59273), 'ng-biomethane': np.int64(4408), 'electric': np.int64(1)}

 {'petrol': np.float64(64.44), 'diesel': np.float64(22.89), 'petrol/electric': np.float64(8.61), 'lpg': np.float64(2.91), 'diesel/electric': np.float64(0.52), 'e85': np.float64(0.38), 'ng': np.float64(0.25), 'ng-biomethane': np.float64(0.02), 'electric': np.float64(0.0)}


Fuel_type
petrol       15380048
diesel        5462712
Name: count, dtype: int64

In [14]:
#Cambiamos las unidades
df1['Engine_size_cm3'] = df1['Engine_size_cm3']/1000
df1.rename(columns={'Engine_size_cm3': 'Engine_size'}, inplace=True)

df1.to_csv('ignore_df_data_clean.csv')

df1

,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,56003309,1710.0,128.0,petrol,2.487,5.7
1,56003313,1710.0,128.0,petrol,2.487,5.7
2,56003314,1710.0,128.0,petrol,2.487,5.7
3,56003351,1635.0,160.0,petrol,1.987,7.1
4,56003352,1635.0,160.0,petrol,1.987,7.1
...,...,...,...,...,...,...
23869025,124068624,1395.0,135.0,petrol,1.498,6.0
23869026,124068626,1387.0,117.0,diesel,1.968,4.5
23869028,124068628,983.0,119.0,petrol,1.197,5.2
23869029,124068629,1665.0,129.0,petrol,2.487,5.7


In [15]:
#El auto que menos combustible consume es de 5.4 l/100km. Si hay autos que consumen menos es porque estan mal catalogados y son hibridos.
df1 = df1[df['Fuel_consumption_(l/100km)'] > 5.4]

#Los vehiculos mas livianos del mercado rondan los 900 kilos, y a su vez los mas pesados rondan los 3000 kilos
df1 = df1[(df1['Mass_(kg)'] >= 900) & (df['Mass_(kg)'] <= 3000)]

df1

C:\Users\matia\AppData\Local\Temp\ipykernel_22028\899938115.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df1[df['Fuel_consumption_(l/100km)'] > 5.4]
C:\Users\matia\AppData\Local\Temp\ipykernel_22028\899938115.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = df1[(df1['Mass_(kg)'] >= 900) & (df['Mass_(kg)'] <= 3000)]


,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,56003309,1710.0,128.0,petrol,2.487,5.7
1,56003313,1710.0,128.0,petrol,2.487,5.7
2,56003314,1710.0,128.0,petrol,2.487,5.7
3,56003351,1635.0,160.0,petrol,1.987,7.1
4,56003352,1635.0,160.0,petrol,1.987,7.1
...,...,...,...,...,...,...
23869017,124068614,1967.0,167.0,diesel,1.969,6.4
23869019,124068617,1291.0,134.0,petrol,1.199,5.9
23869024,124068623,1395.0,139.0,petrol,1.199,6.1
23869025,124068624,1395.0,135.0,petrol,1.498,6.0


---

### <p style='color: green; text-align: center;'>Vamos a utilizar 1 millon de filas para que el archivo pese menos de 100 megas y poder subirlo a GitHub</p>

In [16]:
# Vamos a utilizar menos volumen de datos para poder maniobrar mas facil
df_aux = df1.sample(2500000) # 2.5 millon random rows
df_aux.to_csv('df_fit.csv', index=False)
print('Tamano del archivo df2:', round((os.path.getsize('df_fit.csv')/1024)/1024, 2), 'MB')

df2 = pd.read_csv('df_fit.csv')
print(df2.shape)
df2

Tamano del archivo df2: 96.09 MB
(2500000, 6)


,ID,Mass_(kg),CO2_emission_(g/km),Fuel_type,Engine_size,Fuel_consumption_(l/100km)
0,123030158,1295.0,132.0,petrol,1.199,5.9
1,50063074,1860.0,193.0,petrol,1.991,8.5
2,480784,1400.0,142.0,petrol,1.498,6.3
3,51827229,1395.0,132.0,petrol,1.499,5.8
4,79992271,1538.0,150.0,petrol,1.598,6.6
...,...,...,...,...,...,...
2499995,85681412,1296.0,131.0,petrol,0.999,5.8
2499996,90308953,1471.0,147.0,petrol,1.498,6.5
2499997,56852404,1720.0,131.0,petrol,2.487,5.8
2499998,124009753,1399.0,138.0,petrol,1.498,6.1


---

#### Convertir el archivo en html
<code>jupyter nbconvert --to html clean_data.ipynb</code>